## Next Steps:

## History
- 19.11.19: removed missing values addition
- 17.11.19: added Depot as bubble
- 17.11.19: 3M Performance now in 20% quantiles
- 16.11.19: initial version

In [76]:
import datetime
import pandas as pd

In [77]:
def floatconv(val):
    try:
        if val.strip():
            return float(val.replace('.','').replace(',','.'))
        else:
            return 0
    except ValueError as ve:
        print("VALUE NOT USABLE for floatconv: #{}#".format(val))

In [78]:
def percentconv(val):
    try:
        if '%' in val:
            return floatconv(val.replace('%', ''))
        else:
            return val
    except ValueError as ve:
        print("VALUE NOT USABLE for percentconv: #{}#".format(val))

In [79]:
intconv = lambda val: 0 if len(str(val)) < 2 else float(str(val).replace('.',''))
converter = {'Aktuell':floatconv, 'Wert in EUR':floatconv, 'Perf. 3 Monate':percentconv, \
             'Perf. 1 Jahr':percentconv, 'Perf. 3 Jahre':percentconv}

In [80]:
long2short = {'Amundi Index Solutio':'Nasdaq',
'Commerzbank AG DAX I':'CobaDax',
'ComStage 1 TecDAX UC':'ComsTecDx',
'Deutsche Telekom AG ':'DTE',
'DWS Aktien Strategie':'DWS AS',
'Fidelity Funds - Eur':'FidEur',
'Fidelity Funds - Glo':'FidGlTec',
'Fidelity Funds - Mal':'FidMal',
'Fidelity Funds - Nor':'FidNordic',
'iShares Automation &':'iShAuto',
'iShares TecDAX(DE)UC':'iShTecDax',
'Lang &#38; Schwarz A':'LS9AJF',
'LBBW Dividenden Stra':'LBBW',
'Nordea 1 Nordic Equi':'Nordea1',
'OEkoworld Klima - C ':'OekoKlima',
'OEkoWorld OEkoVision':'Oekovision',
'SAP SE Inhaber-Aktie':'SAP',
'SEB Nordic Small Cap':'NorSmallCap',
'WisdomTree Brent Cru':'Brent',
'Xtrackers MSCI World':'XtrMinVol',
'Xtrackers MSCI World':'XtrMSCIMomentum'}
def shortname(longname):
    return long2short[longname]

In [81]:
#filename = "musterdepot_Komplett_meineuebersicht_20191112_1043.csv"
filename = "musterdepot_Komplett_meineuebersicht.csv"
data = pd.read_csv(filename, sep=";", header=2, encoding="iso-8859-1", converters = converter, usecols=[0,1,4,19,20,21])
data['Wert']=data['Stück']*data['Aktuell']
data['Name'] = data['Bezeichnung'].apply(lambda x: shortname(x[0:20]))

In [82]:
# add missing values: (better than adding missing values is selecting a better stock exchange)
#Nasdaq Performance
#data.loc[0,'Perf. 3 Jahre'] = 61
#TecDax Performance
#data.loc[2,'Perf. 3 Jahre'] = 75
#A2ANH0 Performance
#data.loc[9,'Perf. 3 Jahre'] = 55

In [83]:
def perf2String(val,quantiles):
    if val < quantiles[0.2]:
        return 'lowest'
    if val < quantiles[0.4]:
        return 'low'
    if val < quantiles[0.6]:
        return 'mid'
    if val < quantiles[0.8]:
        return 'high'
    return 'highest'

In [84]:
quantiles = data['Perf. 3 Monate'].quantile([0.2,0.4,0.6,0.8])
data['Perf3MString'] = data['Perf. 3 Monate'].apply(lambda x: perf2String(x,quantiles))

In [85]:
data['Value3MAgo'] = data['Wert']/(1+data['Perf. 3 Monate']/100)
data['Value1YAgo'] = data['Wert']/(1+data['Perf. 1 Jahr']/100)
data['Value3YAgo'] = data['Wert']/(1+data['Perf. 3 Jahre']/100)

valuetoday = data['Wert'].sum()
v3mago = data['Value3MAgo'].sum()
v1yago = data['Value1YAgo'].sum()
v3yago = data['Value3YAgo'].sum()
p3m = (valuetoday/v3mago-1)*100
p1y = (valuetoday/v1yago-1)*100
p3y = (valuetoday/v3yago-1)*100
print("Portfolio total value: {0:7.2f}, 3-month-performance: {1:3.2f}% , 1Y-perf: {2:3.2f}%, 3Y-perf: {3:3.2f}%"
      .format(valuetoday,p3m,p1y,p3y))

Portfolio total value: 311225.28, 3-month-performance: 9.88% , 1Y-perf: 12.79%, 3Y-perf: 33.23%


In [86]:
data

,Stück,Bezeichnung,Aktuell,Perf. 3 Monate,Perf. 1 Jahr,Perf. 3 Jahre,Wert,Name,Perf3MString,Value3MAgo,Value1YAgo,Value3YAgo
0,40,Amundi Index Solutions NASDAQ 100 UCITS ETF - ...,83.580,9.88,25.56,68.74,3343.200,Nasdaq,mid,3042.591918,2662.631411,1981.272964
1,250,Commerzbank AG DAX Indexzert.(2010/unlim.),133.160,14.08,15.94,23.53,33290.000,CobaDax,highest,29181.276297,28713.127480,26948.919291
2,130,ComStage 1 TecDAX UCITS ETF - I EUR DIS,23.990,10.21,15.49,66.90,3118.700,ComsTecDx,high,2829.779512,2700.406962,1868.603954
3,699,Deutsche Telekom AG Namens-Aktien o.N.,15.210,2.31,0.79,5.64,10631.790,DTE,lowest,10391.740788,10548.457188,10064.170769
4,25,DWS Aktien Strategie Deutschland - LC EUR ACC,433.850,13.63,16.26,28.41,10846.250,DWS AS,highest,9545.234533,9329.305006,8446.577369
5,1800,Fidelity Funds - European Growth Fund - A EUR DIS,16.400,9.27,11.30,19.63,29520.000,FidEur,mid,27015.649309,26522.911051,24676.084594
6,425,Fidelity Funds - Global Technology Fund - Y EU...,34.250,17.48,33.18,82.82,14556.250,FidGlTec,highest,12390.406878,10929.756720,7962.066514
7,200,Fidelity Funds - Malaysia Fund - A USD DIS,35.620,2.78,0.00,6.54,7124.000,FidMal,lowest,6931.309593,7124.000000,6686.690445
8,540,Fidelity Funds - Nordic Fund - A SEK DIS,126.520,9.22,3.57,17.56,68320.800,FidNordic,mid,62553.378502,65965.820218,58115.685607
9,470,iShares Automation &#38; Robotics UCITS ETF - ...,7.394,14.42,28.44,56.03,3475.180,iShAuto,highest,3037.213774,2705.683588,2227.251170


In [87]:
def rd(val):
    return int(round(val))

In [88]:
#['Name','3JPerf','1J Perf','3M Perf','EUR'],
#['Nasdaq',32,22,'high',3214],

#lines with values:
values=""
for i, (index, row) in enumerate(data.iterrows()):
    values += "['"+row['Name']+"'," \
    +str(rd(row['Perf. 3 Jahre']))+"," \
    +str(rd(row['Perf. 1 Jahr']))+",'" \
    +row['Perf3MString']+"'," \
    +str(rd(row['Wert']))+"]," \
    +'\n'

#Depot line with 1y and 3y performance and 3m as part of the name (special color)    
values += "['Depot 3M:"+str(round(p3m,2))+"%'," \
+str(rd(p3y))+"," \
+str(rd(p1y))+"," \
+"'Depot','" \
+str(50000) \
+"']\n"

In [89]:
# read template, replace placeholder and write output file:
with open('portfolioPerformance_in.html','rt') as fin, open('portfolioPerformance.html','wt') as fout:
    for line in fin:
        if '#$0' in line:
            line = line.replace('#$0',values) 
        if '#$1' in line:
            today = datetime.date.today()
            line = line.replace('#$1',today.strftime('%d.%m.%Y'))
        if '#$2' in line:
            line = line.replace('#$2',str(quantiles.values))
        fout.write(line)